In [3]:
import regex
import string
from nltk.corpus import wordnet

import numpy as np
import pandas as pd
from utils.contractions import contractions_dict
from joblib import Parallel, delayed

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
from utils.timer import Timer
import regex

In [4]:
import utils.preprocess as preprocess

In [5]:
filepath ='/home/yiyi/Documents/masterthesis/CPD/data/cleand_query_output_csv/fc73c5ad-3748-4d31-8ab2-014c875b05d7'

- Hilton miami downtown
- 2018-01-17T16:25:00+02:00
- 527 rooms
- 35 million renovation
- mammoth shopping complex, movie theater , 
- guest rooms, meeting space, restuarant, lobby, loading dock, park ,

In [6]:
lemmas, sents, dates, scores = preprocess.get_data(filepath)

Elapsed time: 27.0161 seconds


In [7]:
sents_ =[]
for sent in sents:
    s = regex.sub(' +', ' ', sent).strip()
    sents_.append(s)

In [8]:

import plac
import random
import pathlib
import cytoolz
import numpy
from keras.models import Sequential, model_from_json
from keras.layers import LSTM, Dense, Embedding, Bidirectional
from keras.layers import TimeDistributed
from keras.optimizers import Adam
import thinc.extra.datasets
from spacy.compat import pickle
import spacy
import pandas as pd

Using TensorFlow backend.


In [9]:
from keras_spacy_SA import *

In [25]:
connectives ="and, also, besides, further, furthermore, too, moreover, in addition, then, of equal importance, equally important," +\
            " another,next, afterward, finally, later, last, lastly, at last, now, subsequently, then, when, soon, thereafter,"+\
            " after a short time, the next week, the next month, the next day, a minute later, in the meantime, meanwhile, on the following day," +\
            " at length, ultimately, presently, " +\
            "first, second, (etc.), finally, hence, next, then, from here on, to begin with, last of all, after, before, as soon as, in the end, gradually" +\
            "above, behind, below, beyond, here, there, to the right, to the left, nearby, opposite, on the other side, in the background, directly ahead, along the wall, as you turn right, at the top, across the hall, at this point, adjacent to"+\
            ",for example, to illustrate, for instance, to be specific, such as, moreover, furthermore, just as important, similarly, in the same way ,"+\
            "as a result, hence, so, accordingly, as a consequence, consequently, thus, since, therefore, for this reason, because of this,"+\
            "to this end, for this purpose, with this in mind, for this reason, for these reasons," +\
            "like, in the same manner, in the same manner way, as so, similarly,"+\
            "but, in contrast, conversely, however, still, nevertheless, nonetheless, yet, and yet, on the other hand, on the contrary, or, in spite of this, actually, in fact,"\
            +"in summary, to sum up, to repeat, briefly, in short, finally, on the whole, therefore, as I have said, in conclusion, as you can see"

In [26]:
conns = [' '+x.replace(')','').replace('(','').strip()+' ' for x in connectives.split(',')] # the connectives in the middle

In [ ]:
conns

In [23]:
sents_[:10]

['we stayed in one of the suites on the th floor',
 'two kids and and two adults',
 'the pool was really fantastic and i can not stress enough how much we all enjoyed it',
 'it has a rubberized bottom and stairs so no scuffed knees or snagged bathing suits',
 'the hotel is very close to all the family attractions we wanted to see sea aquarium jungle island but my daughter loved the pool so much she could not wait to leave the attractions to go swimming again',
 'also a porter noticed one day that my daughter looked a little down as we were getting into the elevator and did a magic trick for her that she is still talking about if you have a family this is the hotel for you',
 'my stay at the hilton in downtown miami was hell from day one',
 'because they were overboked my friends and i had to wait and come back later to get the rooms we had reserved for weeks ahead of time',
 'because the person who checked us in the manager at that messed up our confirmation numbers we had to check out

In [33]:
from itertools import chain

In [34]:
s = []
for sent in sents_:
    phrase=[]
    for conn in conns:
        if conn in sent:
            ph = [x.strip() for x in sent.split(conn) if x.strip()!='']
            phrase.append(ph)
    if phrase==[]:
        s.append(sent)
    else:
        p = list(set(chain.from_iterable(phrase)))
        s.append(p)

In [ ]:
s

In [ ]:
s

In [ ]:
nlp = spacy.load("en_vectors_web_lg")
nlp.add_pipe(nlp.create_pipe("sentencizer"))

##### Applying Sentiment Analyser
nlp.add_pipe(SentimentAnalyser.load('lstm_model/', nlp, max_length=128))
##### Getting the Polarity Score
# "docs" is a list which contains sentences we want to classify
sentiments = []
for doc in nlp.pipe(sents_):
    sentiments.append(doc.sentiment)

In [16]:
sent_df = pd.DataFrame(zip(lemmas, sents_, sentiments, dates, scores), columns=['lemma', 'sentence', 'sentiment', 'date', 'score' ])

In [17]:
sent_df.head()

,lemma,sentence,sentiment,date,score
0,stayed one suite floor,we stayed in one of the suites on the th floor,-0.008772,2009-08-01 00:00:00,100
1,two kid two adult,two kids and and two adults,0.015965,2009-08-01 00:00:00,100
2,pool really fantastic stress enough much enjoyed,the pool was really fantastic and i can not st...,0.490455,2009-08-01 00:00:00,100
3,rubberized bottom stair scuffed knee snagged b...,it has a rubberized bottom and stairs so no sc...,-0.485606,2009-08-01 00:00:00,100
4,hotel close family attraction wanted see sea a...,the hotel is very close to all the family attr...,0.482819,2009-08-01 00:00:00,100


In [30]:
for sent, sentiment, score, date in zip(sents_, sentiments, scores, dates):
    if sentiment >=0.2 and 'room' in sent and score <50:
        print(sent,sentiment, score, date)

on the bright side the room was nice with a beautiful view of the islands and very comfortable but service like i mention here is very very poor 0.47164827585220337 40.0 2010-06-18 00:00:00
the tv in the room was ancient 0.44547075033187866 20.0 2010-12-18 00:00:00
we finally got to our room which looked nice when we walked in 0.48888856172561646 40.0 2008-09-24 00:00:00
great room for the money first when we arrived our rooms were not ready then when we returned they had given them away 0.48932284116744995 20.0 2009-07-06 00:00:00
the tv in the room was ancient 0.44547075033187866 20.0 2010-12-18 00:00:00
this theme continues into the room 0.4307791590690613 20.0 2008-11-17 00:00:00
the hotel room was clean 0.4981542229652405 20.0 2009-07-22 00:00:00
when i returned my room was ready about minutes before 0.35833144187927246 20.0 2009-07-22 00:00:00
terable housekeeping staff check in was very nice after being in the room for sometime my elderly mother in law reached under the pillow t